# Modelo de Classificação


Nesse notebook faremos analises referentes a modelos preditivos de classificação, onde nossa variável alvo será FTR (full time result)

## Entendendo as variáveis do dataset

## 🏟️ Informações da Partida
* **Div** — Liga / divisão da partida.  
* **Date** — Data da partida (dd/mm/yy).  
* **Time** — Horário do início do jogo.  
* **HomeTeam** — Time mandante.  
* **AwayTeam** — Time visitante.  
* **Referee** — Árbitro da partida.

---

## ⚽ Placar e Resultados

* **FTHG** — Gols do mandante no fim do jogo.  
* **FTAG** — Gols do visitante no fim do jogo.  
* **FTR / Res** — Resultado final:  
  * `H` — vitória do mandante  
  * `D` — empate  
  * `A` — vitória do visitante  

* **HTHG** — Gols do mandante no intervalo.  
* **HTAG** — Gols do visitante no intervalo.  
* **HTR** — Resultado do intervalo (`H`, `D`, `A`).

---

## 💸 Odds – Bet365

* **B365H** — Odd vitória mandante.  
* **B365D** — Odd empate.  
* **B365A** — Odd vitória visitante.

---

## 💸 Odds – Pinnacle

* **PSH** — Pinnacle odd home.  
* **PSD** — Pinnacle odd draw.  
* **PSA** — Pinnacle odd away.


## 💸 Odds – Mercado (médias)

* **AvgH** — Média de odds home.  
* **AvgD** — Média de odds draw.  
* **AvgA** — Média de odds away.

---

## 🔢 Linhas de Gols – Over/Under

* **B365>2.5** — Bet365 over 2.5 gols.  
* **B365<2.5** — Bet365 under 2.5 gols.

* **P>2.5** — Pinnacle over 2.5 gols.  
* **P<2.5** — Pinnacle under 2.5 gols.

* **Max>2.5** — Maior odd over 2.5.  
* **Max<2.5** — Maior odd under 2.5.

* **Avg>2.5** — Média das odds over 2.5.  
* **Avg<2.5** — Média das odds under 2.5.

---
## 🥗 Adicionais
* **Home_Streak** - Sequencia de vitórias (positivo) e derrotas (negativo)  
* **Away_Streak** - Sequencia de vitórias (positivo) e derrotas (negativo)  
* **Home_CumPoints** - Pontos acumulados na temporada mandante  
* **Away_CumPoints** - Pontos acumulados na temporada visitante  
* **Home_WinRate5** - Taxa de vitória nos últimos 5 jogos mandante  
* **Away_WinRate5** - Taxa de vitória nos últimos 5 jogos visitante  
* **Home_GFGA5** - Taxa de Gols Feitos por Gols sofridos mandante  
* **Away_GFGA5** - Taxa de Gols Feitos por Gols sofridos visitante  
* **Diff_CumPoints** - Diferença dos pontos acumulados  
* **B365_gap** - Diferença entre as odds (visitante - mandante) da bet365  
* **PS_gap** - Diferença entre as odds (visitante - mandante) da pinnacle  
* **PSH_prob_norm** - Odd convertida em probabilidade - prob vitória mandante pinnacle  
* **B365H_prob_norm** - Odd convertida em probabilidade - prob vitória mandante b365


# Importar Bibliotecas 

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

pd.set_option('display.max_columns', None)

# File Paths 

In [3]:
RAW_DATA_PATH = r'..\data\premier_completo_19_25.csv'
EXPANDED_DATA_PATH = r'..\data\premier_completo_expandido_19_25.csv'

# Carregar Dados 

In [4]:
df_raw = pd.read_csv(RAW_DATA_PATH)
print("\nPrimeiras linhas do conjunto de dados:")
df_raw.head(10)


Primeiras linhas do conjunto de dados:


,Div,Season,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,PSH,PSD,PSA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA
0,E0,19,09/08/2019,20:00,Liverpool,Norwich,4,1,H,4,0,H,M Oliver,15,12,7,5,9,9,11,2,0,2,0,0,1.14,10.00,19.00,1.15,9.59,18.05,1.16,10.00,23.00,1.14,8.75,19.83,1.40,3.00,1.40,3.11,1.45,3.11,1.41,2.92,1.96,1.94,1.97,1.95,1.97,2.00,1.94,1.94
1,E0,19,10/08/2019,12:30,West Ham,Man City,0,5,A,0,1,A,M Dean,5,14,3,9,6,13,1,1,2,2,0,0,12.00,6.50,1.22,11.68,6.53,1.26,13.00,6.75,1.29,11.84,6.28,1.25,1.44,2.75,1.49,2.77,1.51,2.77,1.48,2.65,2.00,1.90,2.02,1.90,2.02,1.92,1.99,1.89
2,E0,19,10/08/2019,15:00,Bournemouth,Sheffield United,1,1,D,0,0,D,K Friend,13,8,3,3,10,19,3,4,2,1,0,0,1.95,3.60,3.60,2.04,3.57,3.90,2.06,3.65,4.00,2.01,3.53,3.83,1.90,1.90,1.96,1.96,2.00,1.99,1.90,1.93,2.01,1.89,2.04,1.88,2.04,1.91,2.00,1.88
3,E0,19,10/08/2019,15:00,Burnley,Southampton,3,0,H,0,0,D,G Scott,10,11,4,3,6,12,2,7,0,0,0,0,2.62,3.20,2.75,2.71,3.31,2.81,2.80,3.33,2.85,2.68,3.22,2.78,2.10,1.72,2.17,1.77,2.20,1.78,2.12,1.73,1.92,1.98,1.93,2.00,1.94,2.00,1.91,1.98
4,E0,19,10/08/2019,15:00,Crystal Palace,Everton,0,0,D,0,0,D,J Moss,6,10,2,3,16,14,6,2,2,1,0,1,3.00,3.25,2.37,3.21,3.37,2.39,3.21,3.40,2.52,3.13,3.27,2.40,2.20,1.66,2.23,1.74,2.25,1.74,2.18,1.70,1.85,2.05,1.88,2.05,1.88,2.09,1.84,2.04
5,E0,19,10/08/2019,15:00,Watford,Brighton,0,3,A,0,1,A,C Pawson,11,5,3,3,15,11,5,2,0,1,0,0,1.90,3.40,4.00,1.98,3.44,4.37,2.00,3.50,4.60,1.94,3.41,4.26,2.10,1.72,2.19,1.76,2.24,1.76,2.16,1.71,1.95,1.95,1.98,1.95,1.98,1.98,1.94,1.94
6,E0,19,10/08/2019,17:30,Tottenham,Aston Villa,3,1,H,0,1,A,C Kavanagh,31,7,7,4,13,9,14,0,1,0,0,0,1.30,5.25,10.00,1.30,5.84,10.96,1.33,5.95,12.00,1.30,5.53,10.51,1.66,2.20,1.64,2.40,1.70,2.40,1.65,2.26,1.97,1.93,1.99,1.93,2.00,2.00,1.93,1.94
7,E0,19,11/08/2019,14:00,Leicester,Wolves,0,0,D,0,0,D,A Marriner,15,8,1,2,3,13,12,3,0,2,0,0,2.20,3.20,3.40,2.21,3.34,3.66,2.29,3.38,3.66,2.22,3.28,3.48,2.20,1.66,2.23,1.74,2.25,1.74,2.17,1.70,1.90,2.00,1.90,2.04,1.95,2.04,1.91,1.98
8,E0,19,11/08/2019,14:00,Newcastle,Arsenal,0,1,A,0,0,D,M Atkinson,9,8,2,2,12,7,5,3,1,3,0,0,4.50,3.75,1.72,4.58,3.93,1.81,4.70,4.00,1.83,4.49,3.82,1.79,1.80,2.00,1.83,2.10,1.83,2.14,1.77,2.07,1.85,2.05,1.86,2.07,1.88,2.08,1.85,2.03
9,E0,19,11/08/2019,16:30,Man United,Chelsea,4,0,H,1,0,H,A Taylor,11,18,5,7,15,13,3,5,3,4,0,0,2.10,3.30,3.50,2.21,3.37,3.63,2.28,3.43,3.63,2.19,3.32,3.49,2.00,1.80,2.05,1.87,2.10,1.87,2.01,1.83,1.90,2.00,1.90,2.04,1.92,2.04,1.89,2.00


In [5]:
df_raw_expanded = pd.read_csv(EXPANDED_DATA_PATH)
print("\nPrimeiras linhas do conjunto de dados:")
df_raw_expanded.sample(10)



Primeiras linhas do conjunto de dados:


,Div,Season,Date,Time,MatchID,HomeTeam,Home_CumPoints,Home_Streak,AwayTeam,Away_Streak,Away_CumPoints,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,PSH,PSD,PSA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,Goal_Diff_Home,Home_Win,B365H_prob,B365D_prob,B365A_prob,PSH_prob,PSA_prob,PSD_prob,B365H_prob_norm,B365D_prob_norm,B365A_prob_norm,PSH_prob_norm,PSA_prob_norm,PSD_prob_norm,Points_Home,Points_Away,B365_gap,PS_gap,Home_WinRate5,Home_GF5,Home_GA5,Home_GFGA5,Away_WinRate5,Away_GF5,Away_GA5,Away_GFGA5
1354,E0,22,2023-02-11,15:00,1354,Leicester,21,1,Tottenham,2,39,4,1,H,3,1,H,M Salisbury,15,11,7,4,14,9,1,5,5,2,0,0,3.40,3.60,2.10,3.46,3.70,2.16,3.52,3.84,2.18,3.39,3.64,2.13,1.73,2.10,1.76,2.17,1.80,2.18,1.75,2.11,2.06,1.87,2.06,1.88,2.08,1.88,2.03,1.85,3,1,0.294118,0.277778,0.476190,0.289017,0.462963,0.270270,0.280624,0.265033,0.454343,0.282727,0.452886,0.264387,3,0,1.30,1.30,0.2,7.0,9.0,0.777778,0.600000,8.0,6.0,1.333333
4,E0,19,2019-08-10,17:30,4,Tottenham,0,0,Aston Villa,0,0,3,1,H,0,1,A,C Kavanagh,31,7,7,4,13,9,14,0,1,0,0,0,1.30,5.25,10.00,1.30,5.84,10.96,1.33,5.95,12.00,1.30,5.53,10.51,1.66,2.20,1.64,2.40,1.70,2.40,1.65,2.26,1.97,1.93,1.99,1.93,2.00,2.00,1.93,1.94,2,1,0.769231,0.190476,0.100000,0.769231,0.091241,0.171233,0.725890,0.179744,0.094366,0.745592,0.088437,0.165971,3,0,-8.70,-9.66,0.0,0.0,0.0,NaN,0.000000,0.0,0.0,NaN
1942,E1,24,2024-08-31,15:00,1942,Millwall,1,0,Sheffield Weds,-2,3,3,0,H,0,0,D,L Doughty,18,4,4,1,16,13,6,6,2,1,0,0,2.38,3.30,2.90,2.44,3.35,3.02,2.50,3.41,3.06,2.42,3.32,2.95,2.08,1.73,2.15,1.73,2.19,1.76,2.11,1.72,2.10,1.80,2.11,1.80,2.13,1.80,2.07,1.78,3,1,0.420168,0.303030,0.344828,0.409836,0.331126,0.298507,0.393406,0.283729,0.322864,0.394274,0.318553,0.287173,3,0,-0.52,-0.58,0.0,5.0,7.0,0.714286,0.333333,4.0,6.0,0.666667
1895,E0,23,2024-05-19,16:00,1895,Crystal Palace,46,2,Aston Villa,0,68,5,0,H,2,0,H,D Bond,15,8,9,2,10,8,2,4,1,4,0,0,1.85,4.33,3.60,1.83,4.37,3.74,1.93,4.50,3.84,1.87,4.28,3.67,1.40,3.00,1.39,3.05,1.42,3.12,1.39,2.96,1.87,2.03,1.84,2.06,1.90,2.06,1.85,2.02,5,1,0.540541,0.230947,0.277778,0.546448,0.267380,0.228833,0.515161,0.220103,0.264736,0.524090,0.256440,0.219470,3,0,-1.75,-1.91,0.8,15.0,4.0,3.750000,0.400000,10.0,7.0,1.428571
82,E0,19,2019-10-19,15:00,82,Chelsea,14,2,Newcastle,1,8,1,0,H,0,0,D,A Marriner,16,5,8,0,9,12,11,0,2,1,0,0,1.30,5.50,10.00,1.32,5.70,10.32,1.34,5.90,12.00,1.31,5.62,10.38,1.57,2.37,1.61,2.46,1.65,2.47,1.59,2.38,1.89,2.01,1.91,2.02,1.95,2.04,1.88,2.00,1,1,0.769231,0.181818,0.100000,0.757576,0.096899,0.175439,0.731870,0.172987,0.095143,0.735572,0.094085,0.170343,3,0,-8.70,-9.00,0.6,14.0,7.0,2.000000,0.200000,3.0,9.0,0.333333
1162,E0,22,2022-08-20,15:00,1162,Everton,0,-2,Nott'm Forest,1,3,1,1,D,0,0,D,A Marriner,19,14,8,5,6,9,6,2,3,2,0,0,1.90,3.60,4.00,2.00,3.45,4.27,2.02,3.64,4.30,1.97,3.48,4.05,2.10,1.72,2.10,1.81,2.12,1.82,2.06,1.78,1.98,1.92,2.00,1.93,2.02,1.94,1.98,1.90,0,0,0.526316,0.277778,0.250000,0.500000,0.234192,0.289855,0.499307,0.263523,0.237171,0.488259,0.228693,0.283049,1,1,-2.10,-2.27,0.0,1.0,3.0,0.333333,0.500000,1.0,2.0,0.500000
2179,E1,24,2024-12-29,12:30,2179,Preston,26,1,Sheffield Weds,0,33,3,1,H,1,0,H,S Gill,14,18,7,3,15,18,3,6,0,2,0,0,2.45,2.90,3.10,2.47,3.12,3.08,2.52,3.22,3.10,2.46,3.09,2.99,2.30,1.62,2.32,1.62,2.33,1.63,2.28,1.60,2.11,1.79,2.11,1.77,2.17,1.80,2.08,1.75,2,1,0.408163,0.344828,0.322581,0.404858,0.324675,0.320513,0.379485,0.320599,0.299916,0.385562,0.309201,0.305237,3,0,-0.65,-0.61,0.4,6.0,4.0,1.500000,0.400000,9.0,6.0,1.500000
2347,E1,24,2025-03-15,15:00,2347,Luton,34,1,Middlesbrough,1,53,0,0,D,0,0,D,F Hallam,19,7,5,0,18,12,9,3,2,3,0,0,2.63,3.30,2.63,2.59,3.48,2.66,2.70,3.57,2.75,2.61,3.41,2.63,1.95,1.90,1.95,1.90,2.00,1.91,1.92,1.88,1.92,1.98,1.91,1.96,1.93,1.99,1.91,1.92,0,0,0.380228,0.303030,0.380228,0.386100,0.375940,0.287356,0.357530,0.284940,0.357530,0.367926,0.358244,0.27383

In [7]:
df_raw_expanded.columns

Index(['Div', 'Season', 'Date', 'Time', 'MatchID', 'HomeTeam',
       'Home_CumPoints', 'Home_Streak', 'AwayTeam', 'Away_Streak',
       'Away_CumPoints', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR',
       'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
       'HR', 'AR', 'B365H', 'B365D', 'B365A', 'PSH', 'PSD', 'PSA', 'MaxH',
       'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5',
       'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'B365AHH',
       'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA',
       'Goal_Diff_Home', 'Home_Win', 'B365H_prob', 'B365D_prob', 'B365A_prob',
       'PSH_prob', 'PSA_prob', 'PSD_prob', 'B365H_prob_norm',
       'B365D_prob_norm', 'B365A_prob_norm', 'PSH_prob_norm', 'PSA_prob_norm',
       'PSD_prob_norm', 'Points_Home', 'Points_Away', 'B365_gap', 'PS_gap',
       'Home_WinRate5', 'Home_GF5', 'Home_GA5', 'Home_GFGA5', 'Away_WinRate5',
       'Away_GF5', 'Away_GA5', 'Away_GFGA5']